In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
geolocation = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_geolocation_dataset.csv')
item = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_order_items_dataset.csv')
category = pd.read_csv('/kaggle/input/brazilian-ecommerce/product_category_name_translation.csv')
review = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_order_reviews_dataset.csv')
payment = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_order_payments_dataset.csv')
product = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_products_dataset.csv')
seller = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_sellers_dataset.csv')
customers = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_customers_dataset.csv')
order = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_orders_dataset.csv')

# Order dataset Feature engineering

In [ ]:
order

In [ ]:
#Change data 3,4,5,6,7 to datetime
date = ['order_purchase_timestamp','order_approved_at','order_delivered_carrier_date','order_delivered_customer_date','order_estimated_delivery_date','order_purchase_timestamp']
for i in date:
  order[i] = pd.to_datetime(order[i])
  
order.info()

- Dataset untuk 'date' ada beberapa yang null

In [ ]:
# Purchase name of the day feature
order['day_of_week'] = order['order_purchase_timestamp'].dt.day_name()
order['day_of_week'].head()

In [ ]:
# Purchase hour feature
order['hour'] = order['order_purchase_timestamp'].dt.hour
order['hour'].head()

In [ ]:
# Purchase delivery time feature
order['delivery_time'] = (order['order_delivered_customer_date'] - order['order_purchase_timestamp']).dt.days
order['delivery_time']

In [ ]:
order

## Selesai Feature engineering

In [ ]:
order_df = order[['order_id','customer_id','order_status','day_of_week','hour','delivery_time','order_purchase_timestamp']]

In [ ]:
order_df

# Customers Dataset

In [ ]:
customers

# Merging customers and order dataset

In [ ]:
cust_ord = pd.merge(customers[["customer_id","customer_unique_id"]],order_df[['order_id','customer_id',
                                                                              'order_status','day_of_week',
                                                                              'hour','delivery_time',
                                                                              'order_purchase_timestamp']],on = "customer_id", how = "left")
cust_ord

In [ ]:
cust_ord.info()

# Review

In [ ]:
review

In [ ]:
review_df = review[['review_id','order_id','review_score']]
review_df

## Merging cust_ord and review dataset

In [ ]:
custord_rev = pd.merge(cust_ord,review_df,on = "order_id", how = "left")
custord_rev

In [ ]:
custord_rev.isna().sum()

In [ ]:
#custord_rev['review_id'].isna())
#"custord_rev.loc[custord_rev['review_id'] == 'null','installment']"
custord_rev.loc[custord_rev['review_id'].isna()]

In [ ]:
custord_rev = custord_rev.drop_duplicates(subset=['order_id'])

In [ ]:
custord_rev.shape

In [ ]:
custord_rev['order_id'].nunique()

In [ ]:
#For flagging we just don't use canceled and unavailable
custord_rev = custord_rev[(custord_rev.order_status != 'canceled') & (custord_rev.order_status != 'unavailable')]

In [ ]:
custord_rev.shape

In [ ]:
custord_rev.dtypes

In [ ]:
custord_rev["order_purchase_timestamp"] = pd.to_datetime(custord_rev["order_purchase_timestamp"])

In [ ]:
from datetime import datetime

custord_rev['order_purchase_timestamp'] = custord_rev['order_purchase_timestamp'].dt.strftime('%Y-%m')

In [ ]:
custord_rev.isna().sum()

In [ ]:
custord_rev['order_status'].value_counts()

In [ ]:
custord_rev2 = custord_rev[~custord_rev["order_purchase_timestamp"].isna()]
custord_rev2.shape

In [ ]:
custord_rev2.columns

In [ ]:
custord_rev3 = custord_rev2.groupby(["customer_unique_id","order_purchase_timestamp"], as_index = False).agg({"customer_id" : ["first"],
                                                                                                               "customer_unique_id" : ["first"],
                                                                                                               "order_id" : ["first"],
                                                                                                               "order_status" : ["first"],
                                                                                                               "day_of_week" : ["first"],
                                                                                                               "hour" : ["first"],
                                                                                                               "delivery_time" : ["first"],
                                                                                                               "order_purchase_timestamp" : ["first"],
                                                                                                               "review_id" : ["first"],
                                                                                                               "review_score" : ["first"]
                                                                                                              
                                                                               
})
custord_rev3.columns = list(map(''.join,custord_rev3.columns.values))

In [ ]:
custord_rev3.shape

# Flagging

In [ ]:
custord_rev3

In [ ]:
custord_rev3.iloc[0,7]

In [ ]:
custord_rev3.iloc[1,0]

In [ ]:
custord_rev3.dtypes

In [ ]:
custord_rev3["order_purchase_timestampfirst"] = pd.to_datetime(custord_rev3["order_purchase_timestampfirst"])

In [ ]:
size = custord_rev3.shape[0]
#temp = pd.DataFrame(columns=['A'])
a = [] # array kosong yaitu temporary data
for i in range (size-1) :
    if custord_rev3.iloc[i,1] == custord_rev3.iloc[i+1,1] :
        a.append((custord_rev3.iloc[i+1,7] - custord_rev3.iloc[i,7])/np.timedelta64(1,'M')) # merubah formatnya, karena nanti berubha jadi days, ini dirubah jadi angka 
    else :
        a.append((custord_rev3.iloc[i,7] - custord_rev3.iloc[i,7])/np.timedelta64(1,'M'))

In [ ]:
diff = pd.DataFrame(a, columns = ["diff"])
diff = round(diff)
diff.head()

In [ ]:
diff.dtypes

In [ ]:
diff.value_counts()

In [ ]:
custord_rev4 = pd.concat([custord_rev3, diff], axis=1)
custord_rev4.head(10)

In [ ]:
custord_rev4["diff"] = custord_rev4["diff"].fillna(0)
custord_rev4

In [ ]:
custord_rev4["diff"].value_counts()

In [ ]:
def renovated(df) : 
    if df["diff"] <= 0 : 
        return 0
    elif df["diff"] <= 1 :
        return 1
    else :
        return 1

    
custord_rev4["flagging"] = custord_rev4.apply(renovated, axis =1 )

In [ ]:
custord_rev4

In [ ]:
custord_rev4['diff'].value_counts()

In [ ]:
custord_rev4['flagging'].value_counts()

In [ ]:
custord_rev4.corr()

In [ ]:
import matplotlib.pyplot as plt
corr = sns.heatmap(custord_rev4.corr(), annot=True)
plt.show()

# Binning Finale Flagging

In [ ]:
if

In [ ]:
import numpy as np
bins = [-np.inf, 3, 6, 12, np.inf]
labels = [0,1,2,3]
custord_rev4['diff_ket'] = pd.cut(custord_rev4['diff'], bins=bins, labels=labels)

In [ ]:
custord_rev4['diff_ket']

In [ ]:
custord_rev4['diff_ket'].value_counts()

In [ ]:
pd.crosstab(custord_rev4['diff_ket'], [custord_rev4['flagging']], rownames=['diff_ket'], colnames=['flagging'])

## Binning hour

In [ ]:
custord_rev4['hourfirst'].value_counts()

In [ ]:
bins = [0, 6, 12, 18, 24]
labels = [0,1,2,3]
custord_rev4['hourfirst_bin'] = pd.cut(custord_rev4['hourfirst'], bins=bins, labels=labels)

In [ ]:
pd.crosstab(custord_rev4['hourfirst_bin'], [custord_rev4['flagging']], rownames=['hourfirst_bin'], colnames=['flagging'])

## Binning day_of_weekfirst

In [ ]:
custord_rev4['day_of_weekfirst'].value_counts()

In [ ]:
def renovated(custord_rev4):
    if custord_rev4['day_of_weekfirst'] == 'Monday': 
        return 0
    elif custord_rev4['day_of_weekfirst'] == 'Tuesday':
        return 1
    elif custord_rev4['day_of_weekfirst'] == 'Wednesday':
        return 2
    elif custord_rev4['day_of_weekfirst'] == 'Thursday':
        return 3
    elif custord_rev4['day_of_weekfirst'] == 'Friday':
        return 4
    elif custord_rev4['day_of_weekfirst'] == 'Saturday':
        return 5
    else:
        return 6
    
custord_rev4['day_of_weekfirst'] = custord_rev4.apply(renovated, axis=1)

In [ ]:
custord_rev4['day_of_weekfirst']

In [ ]:
pd.crosstab(custord_rev4['day_of_weekfirst'], [custord_rev4['flagging']], rownames=['day_of_weekfirst'], colnames=['flagging'])

## Binning delivery_timefirst

In [ ]:
custord_rev4['delivery_timefirst']

In [ ]:
custord_rev4.loc[(custord_rev4['delivery_timefirst'] <= 10)].shape

In [ ]:
#custord_rev4.loc([custord_rev4['delivery_timefirst'] <= 10]).value_counts
#custord_rev4.loc[(custord_rev4['Country'] <= 10)].value_counts()

In [ ]:
n, bins, patches = plt.hist(custord_rev4['delivery_timefirst'], 10, density=True, facecolor='g', alpha=0.75)


plt.xlabel('Smarts')
plt.ylabel('Probability')
plt.title('Histogram of IQ')
plt.text(60, .025, r'$\mu=100,\ \sigma=15$')
plt.xlim(0,10)
plt.ylim(0, 0.03)
plt.grid(True)
plt.show()

In [ ]:
bins = [0, 1, 4, 7, 14, np.inf]
labels = [0,1,2,3,4]
custord_rev4['delivery_timefirst_bin'] = pd.cut(custord_rev4['delivery_timefirst'], bins=bins, labels=labels)
custord_rev4['delivery_timefirst_bin']

In [ ]:
pd.crosstab(custord_rev4['delivery_timefirst'], [custord_rev4['flagging']], rownames=['delivery_timefirst'], colnames=['flagging'])

## WoE dan IV
Hasil dari WoE dan IV tidak ada feature yang cukup memiliki bobot untuk menjadi predictor

# Logistic Regression

In [ ]:
custord_rev4.columns

In [ ]:
df = custord_rev4[['day_of_weekfirst', 'hourfirst','delivery_timefirst', 'order_purchase_timestampfirst',
                   'review_scorefirst', 'diff', 'flagging', 'diff_ket', 'hourfirst_bin',
                   'delivery_timefirst_bin']]

In [ ]:
df

# Feature enggineering order_purchase_timestampfirst

In [ ]:
df['year'] = df['order_purchase_timestampfirst'].dt.year
df['month'] = df['order_purchase_timestampfirst'].dt.month

In [ ]:
pd.crosstab(df['month'], [df['flagging']], rownames=['month'], colnames=['flagging'])

In [ ]:
pd.crosstab(df['year'], [df['flagging']], rownames=['year'], colnames=['flagging'])

IV of Year is 0,22 that have medium predictor

In [ ]:
df_non_target = pd.factorize(df['year'])[0].reshape(-1, 1) 
df_target =  df['flagging']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_non_target, df_target, random_state=100, test_size=0.3)
print("Jumlah data training: ", X_train.shape)
print("Jumlah data testing: ", X_test.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression

# all parameters not specified are set to their defaults
logisticRegr = LogisticRegression()

In [ ]:
logisticRegr.fit(X_train, y_train)

In [ ]:
# Buat prediksi pada test dataset
from sklearn.metrics import accuracy_score
ypred_log=logisticRegr.predict(X_test)

# Menghitung akurasi data
print("Test accuracy: ", accuracy_score(y_test,ypred_log))

In [ ]:
score = logisticRegr.score(X_test, y_test)
print(score)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics

In [ ]:
print(metrics.confusion_matrix(y_test, ypred_log), '\n')
print(metrics.classification_report(y_test, ypred_log))

Hasil prediksi sangat buruk, hanya dapat memprediksi churn, tidak dapat memprediksi sama sekali yang non-churn

## Dengan feature Review score, Hour, Month, Year Sebagai 4 feature WoE dan IV tertinggi

In [ ]:
df2 = df[['hourfirst','month','year','flagging']] 

In [ ]:
df2.isna().sum()

In [ ]:
df_f1 = df2[['hourfirst','month','year']]  #df_feature1
df_l1 = df2['flagging']                    #df_label1

In [ ]:
from sklearn.model_selection import train_test_split

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(df_f1, df_l1, random_state=100, test_size=0.5)
print("Jumlah data training: ", X_train_2.shape)
print("Jumlah data testing: ", X_test_2.shape)

In [ ]:
logisticRegr.fit(X_train_2, y_train_2)

In [ ]:
ypred_log=logisticRegr.predict(X_test_2)

In [ ]:
print(metrics.confusion_matrix(y_test_2, ypred_log), '\n')
print(metrics.classification_report(y_test_2, ypred_log))

Hasilnya masih sama

# NN

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
# Membagi dataIv menjadi feature (x) & target (y)
x_nn=df2[['hourfirst','month','year']]
y_nn=df2['flagging']

In [ ]:
# Split dataset
x_train_nn, x_test_nn, y_train_nn, y_test_nn = train_test_split(x_nn, y_nn, test_size=0.3, random_state=50) 

# Membuat model object dan menyimpannya dalam variabel clf
clf = MLPClassifier(hidden_layer_sizes=(6,5),
                    random_state=10,
                    verbose=True,
                    learning_rate_init=0.01)

# Menyesuaikan (fitting) data dengan model
clf.fit(x_train_nn,y_train_nn)

# Buat prediksi pada test dataset
ypred_nn=clf.predict(x_test_nn)

# Menghitung akurasi data
print("Test accuracy: ", accuracy_score(y_test_nn,ypred_nn))

In [ ]:
print(metrics.confusion_matrix(y_test_nn, ypred_nn), '\n')
print(metrics.classification_report(y_test_nn, ypred_nn))

- Hasil machine learning tidak dapat memprediksi customer non-churn